In [1]:
from src.features.build_features import handle_zeros, resample_KL
from src.visualization.visualize import plot_divergence
from src.data.make_dataset import preprocess
from pathlib import Path
import pandas as pd


In [2]:
FILE_NAME = 'uniform_date.csv'
PATH = Path('../data/raw')
cycle3 = pd.read_csv(PATH / FILE_NAME)
processed = preprocess(cycle3)
processed = handle_zeros(processed)

/Users/zarak/.local/share/virtualenvs/focus_plots-vLdWlo3v/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



> For the group of timestamps that you are considering for each bubble, we want to calculate the mean for each bin mu_b where b is the bin. Then for each forecast f_t_b, where t is the index of the timestamp, we are going to calculate the KL-divergence
K_t = sum(mu_b * log2(f_t_b / mu_b)) along the b axis
Finally we are going to calculate the mean forKL, mean(KL_t) along the t axis.

Sorry for the dumb questions, but I'm not quite clear which two discrete probability distributions we are computing the KL divergence for. I assume that f_t_b the forecast for a particular forecaster at time t and bin b? So I also have to condition on the forecasters, right? In this case, we can compute the KL divergence across the entire dataset and then aggregate over a time interval. This mean value for each time interval will represent the size of a bubble.

There are no dumb questions; I asked so many that must be obvious to you :-)
First of all I should have written p_t_b, because I've been trying to use p in my documentation for the probability guessed by a forecaster, and f the frequency of the real event, aka actual. I don't think you have to condition on the forecaster, (only the team, mango or kiwi). I intentionally used indexes _t_b, without any index for the forecaster, because we want to see the dispersion (divergence) of all forecasts in a certain time interval, compared to the average of the group. So we will have to compute means for the group, during that interval, then we will compute the divergences from that mean for each forecast in the time interval, and then we will average the divergences. 

I'll work on mean and KL taking as input beginning and ending time stamp. Did you do anything to condition time stamps? I see that they are in two different formats. In the spreadsheets I used before, I solved that by changing the format for the whole column.


**So we will have to compute means for the group, during that interval, then we will compute the divergences from that mean for each forecast in the time interval, and then we will average the divergences.**

In [3]:
question = 'CFF 304.4'
resampling_period = 'H'

In [42]:
processed

,Forecaster,Forecaster.ID,Question,Forecast,TeamName,Commuity.ID,Ordered.Bin.Number,Number.of.Bins,True.Answer,Weight,WRPS,CrossEntropy,FairSkill,SWRPS,Final,Initial,Process.Gain,Tvalue
Uniform Date Format,,,,,,,,,,,,,,,,,,
2019-11-11 22:00:34,Dimitry Sarin,71,CFF 307.1,0.700,Kiwi,1,1,2,0.60,1.00,0.01,-0.695,-0.0020,0.1000,0,1,0.1500,43780.91706
2019-11-11 22:00:34,Dimitry Sarin,71,CFF 307.1,0.300,Kiwi,1,2,2,0.40,1.00,0.01,-0.695,-0.0020,0.1000,0,1,0.1500,43780.91706
2019-11-11 22:02:20,Dimitry Sarin,71,CFF 308.1,0.300,Kiwi,1,2,2,0.93,1.00,0.3969,-1.143,-0.4500,0.6300,0,1,-0.3700,43780.91829
2019-11-11 22:02:20,Dimitry Sarin,71,CFF 308.1,0.700,Kiwi,1,1,2,0.07,1.00,0.3969,-1.143,-0.4500,0.6300,0,1,-0.3700,43780.91829
2019-11-11 22:02:59,Dimitry Sarin,71,CFF 308.3,0.005,Kiwi,1,1,5,0.01,2.52,0.1155,-1.599,0.0106,0.3398,0,1,-0.0770,43780.91874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-25 04:44:00,David Smerdon,87,CFF 304.4,0.005,Mango,2,1,5,0.13,2.52,0.1143,-2.363,-0.7540,0.3381,1,0,0.2497,NaN
2019-11-25 04:44:00,David Smerdon,87,CFF 304.4,0.050,Mango,2,2,5,0.22,2.52,0.1143,-2.363,-0.7540,0.3381,1,0,0.2497,NaN
2019-11-25 04:44:00,David Smerdon,87,CFF 304.4,0.350,Mango,2,3,5,0.32,2.52,0.1143,-2.363,-0.7540,0.3381,1,0,0.2497,NaN


- The KL-divergence I had in mind is different. First we compute the average of the forecasts in that time window, then we compute for each forecaster the divergence of that forecaster from that time window average. I also prefer to vectorize that operation as I did in DTC, because I get a feel for the dimensions involved.
- vector{Ave_i} dot.product ( log2( {matrix forecasters of shape (m,n)} / vector{Ave_i} ) )
- The division is done by bin with broadcast to all n forecasters. Then when you have all the KL_divergences for each forecaster, you average them to obtain the bubble size for that time window.
- The number you calculated, I believe is sort of an entropy for the average.


Notes:
1. $m$ is the number of bins
2. $n$ is the number of forecasts
3. $i$ is the index of the timestamp

In [133]:
def KL_div(df):
    average_forecast = df.groupby(['Question', 'Ordered.Bin.Number']).Forecast.mean()
    forecasts = df.pivot_table(values='Forecast', index='Ordered.Bin.Number', columns='Uniform Date Format')
    print(forecasts)

In [134]:
# def KL_div(df):
#     return df.groupby(['Forecaster', 'Question', 'Ordered.Bin.Number']).Forecast.mean()

In [135]:
processed.head(2)

,Forecaster,Forecaster.ID,Question,Forecast,TeamName,Commuity.ID,Ordered.Bin.Number,Number.of.Bins,True.Answer,Weight,WRPS,CrossEntropy,FairSkill,SWRPS,Final,Initial,Process.Gain,Tvalue
Uniform Date Format,,,,,,,,,,,,,,,,,,
2019-11-11 22:00:34,Dimitry Sarin,71,CFF 307.1,0.7,Kiwi,1,1,2,0.6,1.0,0.01,-0.695,-0.002,0.1,0,1,0.15,43780.91706
2019-11-11 22:00:34,Dimitry Sarin,71,CFF 307.1,0.3,Kiwi,1,2,2,0.4,1.0,0.01,-0.695,-0.002,0.1,0,1,0.15,43780.91706


In [136]:
processed.groupby(pd.Grouper(freq='60Min', label='right', level='Uniform Date Format')).apply(KL_div).head(60)

Uniform Date Format  2019-11-11 22:00:34  2019-11-11 22:02:20  \
Ordered.Bin.Number                                              
1                                    0.7                  0.7   
2                                    0.3                  0.3   
3                                    NaN                  NaN   
4                                    NaN                  NaN   
5                                    NaN                  NaN   
6                                    NaN                  NaN   
7                                    NaN                  NaN   

Uniform Date Format  2019-11-11 22:02:59  2019-11-11 22:03:38  \
Ordered.Bin.Number                                              
1                                  0.005                0.005   
2                                  0.005                0.460   
3                                  0.040                0.500   
4                                  0.460                0.040   
5                       

Uniform Date Format  2019-11-12 07:02:30  2019-11-12 07:08:14  \
Ordered.Bin.Number                                              
1                                   0.10                0.005   
2                                   0.25                0.050   
3                                   0.36                0.200   
4                                   0.22                0.450   
5                                   0.07                0.300   

Uniform Date Format  2019-11-12 07:16:30  2019-11-12 07:30:56  \
Ordered.Bin.Number                                              
1                                   0.09                0.450   
2                                   0.41                0.410   
3                                   0.41                0.110   
4                                   0.06                0.030   
5                                   0.03                0.005   

Uniform Date Format  2019-11-12 07:43:55  
Ordered.Bin.Number                        
1 

Uniform Date Format  2019-11-12 16:05:27  2019-11-12 16:34:54  \
Ordered.Bin.Number                                              
1                                    0.7                 0.01   
2                                    0.3                 0.04   
3                                    NaN                 0.06   
4                                    NaN                 0.09   
5                                    NaN                 0.10   
6                                    NaN                 0.50   
7                                    NaN                 0.20   

Uniform Date Format  2019-11-12 16:49:38  2019-11-12 16:51:30  
Ordered.Bin.Number                                             
1                                  0.005                0.005  
2                                  0.290                0.005  
3                                  0.640                0.010  
4                                  0.070                0.390  
5                             

Uniform Date Format  2019-11-13 00:07:02  2019-11-13 00:12:00  \
Ordered.Bin.Number                                              
1                                  0.270                 0.08   
2                                  0.460                 0.40   
3                                  0.180                 0.40   
4                                  0.090                 0.08   
5                                  0.005                 0.04   
6                                    NaN                  NaN   
7                                    NaN                  NaN   

Uniform Date Format  2019-11-13 00:14:54  2019-11-13 00:31:50  \
Ordered.Bin.Number                                              
1                                  0.020                0.005   
2                                  0.050                0.010   
3                                  0.440                0.100   
4                                  0.340                0.420   
5                       

Uniform Date Format  2019-11-13 06:01:02  2019-11-13 06:01:46  \
Ordered.Bin.Number                                              
1                                   0.04                 0.03   
2                                   0.09                 0.04   
3                                   0.18                 0.06   
4                                   0.36                 0.25   
5                                   0.33                 0.62   
6                                    NaN                  NaN   
7                                    NaN                  NaN   

Uniform Date Format  2019-11-13 06:07:05  2019-11-13 06:09:24  \
Ordered.Bin.Number                                              
1                                   0.75                 0.45   
2                                   0.17                 0.33   
3                                   0.05                 0.16   
4                                   0.02                 0.04   
5                       

...

Uniform Date Format  2019-11-24 01:00:18  2019-11-24 01:03:28  \
Ordered.Bin.Number                                              
1                                  0.440                0.430   
2                                  0.520                0.530   
3                                  0.040                0.040   
4                                  0.005                0.005   
5                                    NaN                  NaN   
6                                    NaN                  NaN   
7                                    NaN                  NaN   

Uniform Date Format  2019-11-24 01:10:36  2019-11-24 01:10:42  \
Ordered.Bin.Number                                              
1                                   0.04                  0.3   
2                                   0.09                  0.7   
3                                   0.09                  NaN   
4                                   0.30                  NaN   
5                       

Uniform Date Format  2019-11-24 07:04:00
Ordered.Bin.Number                      
1                                   0.01
2                                   0.17
3                                   0.68
4                                   0.14
Empty DataFrame
Columns: []
Index: []
Uniform Date Format  2019-11-24 09:50:00  2019-11-24 09:52:00  \
Ordered.Bin.Number                                              
1                                    0.5                 0.05   
2                                    0.5                 0.25   
3                                    NaN                 0.50   
4                                    NaN                 0.20   
5                                    NaN                  NaN   

Uniform Date Format  2019-11-24 09:53:00  2019-11-24 09:54:00  \
Ordered.Bin.Number                                              
1                                  0.005                0.050   
2                                  0.005                0.400   


Uniform Date Format  2019-11-24 16:00:22  2019-11-24 16:10:35  \
Ordered.Bin.Number                                              
1                                   0.34                 0.49   
2                                   0.66                 0.51   
3                                    NaN                  NaN   
4                                    NaN                  NaN   
5                                    NaN                  NaN   

Uniform Date Format  2019-11-24 16:16:47  2019-11-24 16:24:45  \
Ordered.Bin.Number                                              
1                                   0.01                 0.01   
2                                   0.27                 0.23   
3                                   0.56                 0.56   
4                                   0.16                 0.20   
5                                    NaN                  NaN   

Uniform Date Format  2019-11-24 16:25:02  2019-11-24 16:26:28  \
Ordered.Bin.Number     

Uniform Date Format  2019-11-24 22:00:00  2019-11-24 22:03:00  \
Ordered.Bin.Number                                              
1                                   0.06                0.005   
2                                   0.22                0.010   
3                                   0.50                0.080   
4                                   0.22                0.370   
5                                    NaN                0.540   

Uniform Date Format  2019-11-24 22:04:00  2019-11-24 22:04:42  \
Ordered.Bin.Number                                              
1                                   0.55                 0.32   
2                                   0.45                 0.40   
3                                    NaN                 0.20   
4                                    NaN                 0.07   
5                                    NaN                 0.01   

Uniform Date Format  2019-11-24 22:05:00  2019-11-24 22:06:45  \
Ordered.Bin.Number     

""
